In [15]:
import pandas as pd
import numpy as np

In [40]:
from sklearn.impute import SimpleImputer

data_uni_2 = pd.read_csv("data_uni.csv")

data_uni_2.iloc[:, [24,25,29]]= data_uni_2.iloc[:, [24,25,29]].fillna(0)
column_miss= list(data_uni_2.columns[data_uni_2.isnull().any()])

# IMPUTER ===========================================================================

imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

data_uni_2[column_miss]= imputer.fit_transform(data_uni_2[column_miss])

data_uni_2= data_uni_2.drop(["Course","AWM year 1", "AWM year 2", "AWM year 3", "Overall AWM", "Progression", "Attendance"], axis = 1)

# FORMATING MANUALLY ================================================================

#ethinicty binarisation 
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("White", "0")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Asian", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Black/Black British African", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Other ethnic background", "1")

# Polar 4 Score Binarisation
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(5, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(4, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(3, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(2, 1)

#splitting X and y ==================================================================
y = data_uni_2["desertion"]
X = data_uni_2.drop(["desertion"], axis = 1)

In [17]:
#data_uni_2.info()

In [41]:
from pycaret.classification import *

classify = setup(data= data_uni_2, target = "desertion", log_experiment = True, log_plots= True, remove_multicollinearity= True, session_id=2610, multicollinearity_threshold=0.8, log_profile= True, fix_imbalance=True, feature_selection = True, feature_selection_threshold=0.8,  pca= True, pca_components=0.95, numeric_features=["First Sit", "Fails", "No Submissions", "Late Submission", "Pass",  "Maths", "UCAS Points"])

,Description,Value
0,session_id,2610
1,Target,desertion
2,Target Type,Binary
3,Label Encoded,"no: 0, yes: 1"
4,Original Data,"(1033, 29)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,19
8,Ordinal Features,False
9,High Cardinality Features,False


In [42]:
compare_models(include=["rf", "ada","gbc"])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9088,0.8206,0.7689,0.6935,0.7212,0.6675,0.6741,0.0450
nb,Naive Bayes,0.9074,0.8428,0.6886,0.7102,0.6940,0.6398,0.6432,0.0120
lr,Logistic Regression,0.9046,0.8724,0.7432,0.6810,0.7025,0.6463,0.6527,0.0140
gbc,Gradient Boosting Classifier,0.8921,0.8346,0.7598,0.6390,0.6846,0.6209,0.6308,0.0520
rf,Random Forest Classifier,0.8852,0.8561,0.7235,0.6160,0.6602,0.5924,0.5986,0.3720
dt,Decision Tree Classifier,0.8590,0.7892,0.6879,0.5452,0.6039,0.5208,0.5286,0.0070


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=2610)

In [20]:
rf= create_model("rf", cross_validation= True)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8904,0.9260,0.7273,0.6154,0.6667,0.6016,0.6046
1,0.8767,0.9167,0.6667,0.6154,0.6400,0.5658,0.5664
2,0.8904,0.9474,0.6667,0.6667,0.6667,0.6011,0.6011
3,0.9167,0.9687,0.9091,0.6667,0.7692,0.7198,0.7327
4,0.9028,0.9471,0.9091,0.6250,0.7407,0.6834,0.7016
5,0.9167,0.8443,0.8182,0.6923,0.7500,0.7004,0.7039
6,0.9167,0.9613,0.9091,0.6667,0.7692,0.7198,0.7327
7,0.8472,0.8860,0.6364,0.5000,0.5600,0.4692,0.4742
8,0.9028,0.9069,0.8182,0.6429,0.7200,0.6622,0.6692


In [21]:
rf_results= pull()
rf_scores =rf_results.loc[["Mean"]]
print(rf_scores)

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
Mean    0.8963  0.9252  0.7788  0.6358  0.6978  0.6361  0.6425


In [22]:
rf= create_model("rf", cross_validation= False)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9161,0.9268,0.7609,0.7,0.7292,0.6797,0.6805


In [23]:
tuned_rf= tune_model(rf, optimize = "F1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8904,0.9223,0.7273,0.6154,0.6667,0.6016,0.6046
1,0.9041,0.9276,0.8333,0.6667,0.7407,0.6828,0.6892
2,0.9178,0.9658,0.9167,0.6875,0.7857,0.7361,0.7478
3,0.9444,0.9762,1.0000,0.7333,0.8462,0.8132,0.8278
4,0.9028,0.9344,0.9091,0.6250,0.7407,0.6834,0.7016
5,0.9167,0.9024,0.8182,0.6923,0.7500,0.7004,0.7039
6,0.9583,0.9732,1.0000,0.7857,0.8800,0.8552,0.8643
7,0.9028,0.9434,0.8182,0.6429,0.7200,0.6622,0.6692
8,0.9306,0.8942,0.9091,0.7143,0.8000,0.7587,0.7668


In [24]:
tuned_rf_results = pull()
tuned_rf_scores = tuned_rf_results.loc[["Mean"]]
print(tuned_rf_scores)

      Accuracy     AUC  Recall   Prec.     F1   Kappa     MCC
Fold                                                         
Mean    0.9212  0.9378  0.8659  0.7052  0.773  0.7262  0.7349


In [25]:
final_rf = calibrate_model(tuned_rf)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8767,0.9296,0.7273,0.5714,0.6400,0.5669,0.5729
1,0.8904,0.9290,0.8333,0.6250,0.7143,0.6482,0.6585
2,0.9041,0.9686,0.9167,0.6471,0.7586,0.7010,0.7176
3,0.9444,0.9732,1.0000,0.7333,0.8462,0.8132,0.8278
4,0.9167,0.9404,0.9091,0.6667,0.7692,0.7198,0.7327
5,0.9167,0.8964,0.8182,0.6923,0.7500,0.7004,0.7039
6,0.9583,0.9836,1.0000,0.7857,0.8800,0.8552,0.8643
7,0.8889,0.9374,0.8182,0.6000,0.6923,0.6265,0.6377
8,0.9306,0.8964,0.9091,0.7143,0.8000,0.7587,0.7668


In [26]:
evaluate_model(final_rf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [27]:
gbc = create_model("gbc",cross_validation= False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.929,0.9368,0.8696,0.7143,0.7843,0.7423,0.7475


In [28]:
gbc = create_model("gbc",cross_validation= True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8630,0.9172,0.7273,0.5333,0.6154,0.5344,0.5440
1,0.8493,0.8641,0.6667,0.5333,0.5926,0.5016,0.5063
2,0.8767,0.9228,0.9167,0.5789,0.7097,0.6364,0.6635
3,0.9444,0.9844,1.0000,0.7333,0.8462,0.8132,0.8278
4,0.9167,0.9523,0.9091,0.6667,0.7692,0.7198,0.7327
5,0.8750,0.9098,0.8182,0.5625,0.6667,0.5930,0.6087
6,0.9306,0.9896,0.9091,0.7143,0.8000,0.7587,0.7668
7,0.8056,0.8890,0.5455,0.4000,0.4615,0.3463,0.3525
8,0.8889,0.8949,0.9091,0.5882,0.7143,0.6492,0.6729


In [29]:
tuned_gbc= tune_model(gbc, optimize= "F1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8904,0.9120,0.7273,0.6154,0.6667,0.6016,0.6046
1,0.8904,0.9146,0.6667,0.6667,0.6667,0.6011,0.6011
2,0.9041,0.9262,0.8333,0.6667,0.7407,0.6828,0.6892
3,0.9306,0.9732,1.0000,0.6875,0.8148,0.7739,0.7944
4,0.9167,0.9493,0.9091,0.6667,0.7692,0.7198,0.7327
5,0.9167,0.9173,0.8182,0.6923,0.7500,0.7004,0.7039
6,0.9583,0.9657,1.0000,0.7857,0.8800,0.8552,0.8643
7,0.9028,0.9210,0.8182,0.6429,0.7200,0.6622,0.6692
8,0.9306,0.9121,0.9091,0.7143,0.8000,0.7587,0.7668


In [30]:
final_gbc = calibrate_model(tuned_gbc)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8904,0.9150,0.7273,0.6154,0.6667,0.6016,0.6046
1,0.9041,0.9085,0.7500,0.6923,0.7200,0.6623,0.6630
2,0.9041,0.9358,0.8333,0.6667,0.7407,0.6828,0.6892
3,0.9167,0.9732,1.0000,0.6471,0.7857,0.7369,0.7638
4,0.9167,0.9463,0.9091,0.6667,0.7692,0.7198,0.7327
5,0.9167,0.8949,0.8182,0.6923,0.7500,0.7004,0.7039
6,0.9583,0.9672,1.0000,0.7857,0.8800,0.8552,0.8643
7,0.8889,0.9225,0.8182,0.6000,0.6923,0.6265,0.6377
8,0.9306,0.9076,0.9091,0.7143,0.8000,0.7587,0.7668


In [31]:
evaluate_model(final_gbc)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [32]:
ada = create_model("ada",cross_validation= False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9258,0.9216,0.8478,0.7091,0.7723,0.7284,0.7325


In [33]:
ada = create_model("ada",cross_validation= True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8493,0.9150,0.8182,0.5000,0.6207,0.5334,0.5586
1,0.8082,0.7999,0.6667,0.4444,0.5333,0.4187,0.4323
2,0.8493,0.8682,0.9167,0.5238,0.6667,0.5785,0.6163
3,0.9306,0.9829,1.0000,0.6875,0.8148,0.7739,0.7944
4,0.8750,0.8972,0.8182,0.5625,0.6667,0.5930,0.6087
5,0.8750,0.9009,0.8182,0.5625,0.6667,0.5930,0.6087
6,0.9583,0.9911,1.0000,0.7857,0.8800,0.8552,0.8643
7,0.8472,0.9225,0.7273,0.5000,0.5926,0.5025,0.5159
8,0.8750,0.8748,0.8182,0.5625,0.6667,0.5930,0.6087


In [34]:
tune_ada = tune_model(ada, optimize="F1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8082,0.9047,0.8182,0.4286,0.5625,0.4546,0.4936
1,0.8356,0.9324,0.9167,0.5000,0.6471,0.5517,0.5948
2,0.9178,0.9577,1.0000,0.6667,0.8000,0.7509,0.7753
3,0.9028,0.9829,1.0000,0.6111,0.7586,0.7021,0.7355
4,0.9028,0.9396,0.9091,0.6250,0.7407,0.6834,0.7016
5,0.9028,0.9188,0.8182,0.6429,0.7200,0.6622,0.6692
6,0.9444,1.0000,1.0000,0.7333,0.8462,0.8132,0.8278
7,0.8333,0.9270,0.8182,0.4737,0.6000,0.5040,0.5341
8,0.8750,0.9009,0.8182,0.5625,0.6667,0.5930,0.6087


In [35]:
final_ada = calibrate_model(tune_ada)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8356,0.9164,0.8182,0.4737,0.6000,0.5056,0.5356
1,0.8630,0.9317,0.9167,0.5500,0.6875,0.6067,0.6391
2,0.9178,0.9556,1.0000,0.6667,0.8000,0.7509,0.7753
3,0.9167,0.9829,1.0000,0.6471,0.7857,0.7369,0.7638
4,0.9028,0.9352,0.9091,0.6250,0.7407,0.6834,0.7016
5,0.9028,0.9136,0.8182,0.6429,0.7200,0.6622,0.6692
6,0.9444,1.0000,1.0000,0.7333,0.8462,0.8132,0.8278
7,0.8333,0.9277,0.8182,0.4737,0.6000,0.5040,0.5341
8,0.8750,0.8994,0.8182,0.5625,0.6667,0.5930,0.6087


In [36]:
evaluate_model(final_ada)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [37]:
# creating models comparison table
#rf scores







In [38]:
list_scores=[ final_ada_scores, final_gbc_scores, final_rf_scores, tune_ada_scores, tuned_gbc_scores, tuned_rf_scores, ada_scores, rf_scores, gbc_scores]


NameError: name 'final_ada_scores' is not defined

In [ ]:
print(rf_scores)

      Accuracy     AUC  Recall   Prec.      F1  Kappa     MCC
Fold                                                         
Mean    0.8977  0.9256  0.7788  0.6405  0.7009   0.64  0.6459


In [ ]:
#final_gbc_scores
#final_rf_scores
#tune_ada_scores
#tuned_gbc_scores
#tuned_rf_scores
#ada_scores
rf_scores
#gbc_scores




,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
Mean,0.8977,0.9256,0.7788,0.6405,0.7009,0.64,0.6459


In [ ]:
final_ada_scores.set_axis(["ada"], axis="index")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
ada,0.8908,0.9413,0.8826,0.6129,0.7177,0.6538,0.6744


In [ ]:
results

NameError: name 'results' is not defined

In [ ]:
print(final_ada_scores)
results

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
Mean    0.8908  0.9413  0.8826  0.6129  0.7177  0.6538  0.6744


,0
0,final_ada_scores
1,tune_ada_scores
